# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from configvacation import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [18]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/csv_Weather/WeatherPy_database.csv")


# Display sample data
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Inirida,CO,3.8653,-67.9239,77.49,88,48,2.06,scattered clouds
1,1,Polyarnyy,RU,69.1989,33.4478,37.89,99,100,6.87,overcast clouds
2,2,Chicoana,AR,-25.1009,-65.5331,61.05,74,6,3.42,clear sky
3,3,Whitehorse,CA,60.7161,-135.0538,57.97,41,100,19.57,overcast clouds
4,4,Edson,CA,53.5835,-116.4356,71.74,28,0,6.91,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [19]:
# Prompt the user to enter minimum and maximum temperature criteria.
# min 55? and max 82
min_temp_criteria = float(input("What is your desired minimum temperature for your trip? "))
max_temp_criteria = float(input("What is your desired maximum temperature for your trip? "))

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [20]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp_criteria) & (city_data_df["Max Temp"] >= min_temp_criteria)
]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Inirida,CO,3.8653,-67.9239,77.49,88,48,2.06,scattered clouds
2,2,Chicoana,AR,-25.1009,-65.5331,61.05,74,6,3.42,clear sky
3,3,Whitehorse,CA,60.7161,-135.0538,57.97,41,100,19.57,overcast clouds
4,4,Edson,CA,53.5835,-116.4356,71.74,28,0,6.91,clear sky
7,7,Bogale,MM,16.2833,95.4000,79.27,89,100,8.61,overcast clouds
11,11,Utrik,MH,11.2278,169.8474,81.03,80,96,15.55,overcast clouds
12,12,Chonchi,CL,-42.6128,-73.8092,55.11,99,100,1.97,overcast clouds
15,15,Margaret River,AU,-33.9500,115.0667,60.13,74,30,14.94,scattered clouds
17,17,Mikomeseng,GQ,2.1361,10.6132,70.11,99,67,0.83,light rain
20,20,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,79.79,94,75,9.22,light intensity drizzle rain


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [21]:
# Any nulls
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [22]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Inirida,CO,3.8653,-67.9239,77.49,88,48,2.06,scattered clouds
2,2,Chicoana,AR,-25.1009,-65.5331,61.05,74,6,3.42,clear sky
3,3,Whitehorse,CA,60.7161,-135.0538,57.97,41,100,19.57,overcast clouds
4,4,Edson,CA,53.5835,-116.4356,71.74,28,0,6.91,clear sky
7,7,Bogale,MM,16.2833,95.4000,79.27,89,100,8.61,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
559,559,Cadale,SO,2.7603,46.3222,80.69,86,64,15.57,broken clouds
560,560,Fuling,CN,29.7022,107.3919,67.03,75,100,2.35,overcast clouds
561,561,Bocaiuva,BR,-17.1078,-43.8150,80.31,58,72,3.13,broken clouds
562,562,Pimampiro,EC,0.4000,-77.9667,64.60,84,100,2.21,light rain


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [23]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# Don't forget () after copy function
hotel_df = clean_travel_cities[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng
0,Inirida,CO,77.49,scattered clouds,3.8653,-67.9239
2,Chicoana,AR,61.05,clear sky,-25.1009,-65.5331
3,Whitehorse,CA,57.97,overcast clouds,60.7161,-135.0538
4,Edson,CA,71.74,clear sky,53.5835,-116.4356
7,Bogale,MM,79.27,overcast clouds,16.2833,95.4000
...,...,...,...,...,...,...
559,Cadale,SO,80.69,broken clouds,2.7603,46.3222
560,Fuling,CN,67.03,overcast clouds,29.7022,107.3919
561,Bocaiuva,BR,80.31,broken clouds,-17.1078,-43.8150
562,Pimampiro,EC,64.60,light rain,0.4000,-77.9667


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [24]:

# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Inirida,CO,77.49,scattered clouds,3.8653,-67.9239,
2,Chicoana,AR,61.05,clear sky,-25.1009,-65.5331,
3,Whitehorse,CA,57.97,overcast clouds,60.7161,-135.0538,
4,Edson,CA,71.74,clear sky,53.5835,-116.4356,
7,Bogale,MM,79.27,overcast clouds,16.2833,95.4000,
11,Utrik,MH,81.03,overcast clouds,11.2278,169.8474,
12,Chonchi,CL,55.11,overcast clouds,-42.6128,-73.8092,
15,Margaret River,AU,60.13,scattered clouds,-33.9500,115.0667,
17,Mikomeseng,GQ,70.11,light rain,2.1361,10.6132,
20,Puerto Baquerizo Moreno,EC,79.79,light intensity drizzle rain,-0.9000,-89.6000,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [25]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [26]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Inirida - nearest hotel: Residencias El Turista del Guainia
Chicoana - nearest hotel: Hostal Los Faroles
Whitehorse - nearest hotel: Town & Mountain Hotel
Edson - nearest hotel: Best Western High Road Inn
Bogale - nearest hotel: No hotel found
Utrik - nearest hotel: No hotel found
Chonchi - nearest hotel: Hotel Huildín
Margaret River - nearest hotel: Margaret River Hotel
Mikomeseng - nearest hotel: Hotel Victor
Puerto Baquerizo Moreno - nearest hotel: Dory's House
Ribeira Grande - nearest hotel: No hotel found
Geraldton - nearest hotel: Ocean Centre Hotel
Miles City - nearest hotel: Historic Olive Hotel
Hamilton - nearest hotel: North Vista Manor
West Island - nearest hotel: Cocos Village Bungalows
Quatre Cocos - nearest hotel: Horisun
Albany - nearest hotel: No hotel found
Guerrero Negro - nearest hotel: Plaza sal paraiso
Yellowknife - nearest hotel: No hotel found
Decorah - nearest hotel: Country Inn & Suites
Adamstown - nearest hotel: No hotel found
Port Mathur

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Inirida,CO,77.49,scattered clouds,3.8653,-67.9239,Residencias El Turista del Guainia
2,Chicoana,AR,61.05,clear sky,-25.1009,-65.5331,Hostal Los Faroles
3,Whitehorse,CA,57.97,overcast clouds,60.7161,-135.0538,Town & Mountain Hotel
4,Edson,CA,71.74,clear sky,53.5835,-116.4356,Best Western High Road Inn
7,Bogale,MM,79.27,overcast clouds,16.2833,95.4000,No hotel found
...,...,...,...,...,...,...,...
559,Cadale,SO,80.69,broken clouds,2.7603,46.3222,No hotel found
560,Fuling,CN,67.03,overcast clouds,29.7022,107.3919,No hotel found
561,Bocaiuva,BR,80.31,broken clouds,-17.1078,-43.8150,No hotel found
562,Pimampiro,EC,64.60,light rain,0.4000,-77.9667,Reyna del Cisne


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [27]:
# Drop the rows where there is no Hotel Name.
# the index are finding the no hotel found data and then using drop to remove it
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df['Hotel Name'] == "No hotel found"].index)
# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Inirida,CO,77.49,scattered clouds,3.8653,-67.9239,Residencias El Turista del Guainia
2,Chicoana,AR,61.05,clear sky,-25.1009,-65.5331,Hostal Los Faroles
3,Whitehorse,CA,57.97,overcast clouds,60.7161,-135.0538,Town & Mountain Hotel
4,Edson,CA,71.74,clear sky,53.5835,-116.4356,Best Western High Road Inn
12,Chonchi,CL,55.11,overcast clouds,-42.6128,-73.8092,Hotel Huildín
...,...,...,...,...,...,...,...
555,Bouar,CF,67.26,overcast clouds,5.9340,15.5960,Mission catholique
556,Fort-Shevchenko,KZ,58.77,overcast clouds,44.5086,50.2630,Akku
557,Banda Aceh,ID,79.95,scattered clouds,5.5577,95.3222,Hotel 61
562,Pimampiro,EC,64.60,light rain,0.4000,-77.9667,Reyna del Cisne


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [28]:
# Create the output File (CSV)
output_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [29]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country)